In [5]:
import GetOldTweets3 as got
import lxml
import pyquery
import requests
import sys
import time
import pandas as pd
import spacy
from datetime import datetime as dt
from dateutil.parser import parse
import calendar

from newsplease.config import CrawlerConfig
from newsplease.config import JsonConfig
from newsplease.helper import Helper
from newspaper import Article

import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

import matplotlib.pyplot as plt

## Define Data Scraper Function

In [7]:
def get_submits(keyword, subreddit, end_utc, start_utc):
    comments=[]
    day = 1
    for i in range(start_utc, end_utc+86400, 86400):
        print (day)
        try:
            res_comm = requests.get(f'https://api.pushshift.io/reddit/search/submission/?q={keyword}&subreddit={subreddit}&after={i}&before=1d&sort=desc&sort_type=score&size=5')
            new_comm_data = res_comm.json()['data']
            #next_utc = new_comm_data[-1]['created_utc'] # grabbing the last comment time
            comments = comments + new_comm_data
            time.sleep(0.3)
        except Exception as e:
            print(e)
            retries += 1
            if retries > 3:
                print('errored but we dont care')
                return comments
                raise e
        day+=1
    return comments        

## Scraping Cryptocurrency related news

In [8]:
start = int(time.mktime(parse('2016-2-1 8:00:00').timetuple()))
end = int(time.mktime(parse('2020-2-24 8:00:00').timetuple()))
print(start, end)
crypto = get_submits('finance', 'cryptocurrency', end, start)

1454335200 1582552800
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [54]:
comm_crypto = pd.DataFrame(crypto)
print(comm_crypto.shape)
print(comm_crypto.info)
comm_crypto.head()

(7398, 78)
<bound method DataFrame.info of                 author author_flair_css_class    author_flair_text brand_safe  \
0        PineappleFund                   noob  redditor for 1 week       True   
1               EaFaer                   None                 None       True   
2              jhamel2                   None                 None       True   
3          amorpisseur                   None                 None       True   
4            ivanraszl                   None                 None        NaN   
...                ...                    ...                  ...        ...   
7393       nugget_alex                   None                 None        NaN   
7394  TheCoreyMatthews                   None                 None        NaN   
7395       nugget_alex                   None                 None        NaN   
7396       nugget_alex                   None                 None        NaN   
7397       nugget_alex                   None                 None

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,edited,full_link,...,allow_live_comments,total_awards_received,author_flair_template_id,awarders,steward_reports,og_description,og_title,media_metadata,author_premium,removed_by_category
0,PineappleFund,noob,redditor for 1 week,True,False,False,1513170057,self.Bitcoin,1.513174e+09,https://www.reddit.com/r/Bitcoin/comments/7jj0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EaFaer,None,None,True,False,False,1512739690,self.Bitcoin,NaN,https://www.reddit.com/r/Bitcoin/comments/7iey...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jhamel2,None,None,True,False,False,1521463923,twitter.com,NaN,https://www.reddit.com/r/Bitcoin/comments/85j8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,amorpisseur,None,None,True,False,False,1515744927,twitter.com,NaN,https://www.reddit.com/r/Bitcoin/comments/7pv8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ivanraszl,None,None,NaN,False,False,1551376036,i.redd.it,NaN,https://www.reddit.com/r/Bitcoin/comments/avtx...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
cols = ['author', 'selftext', 'retrieved_on', 'score']
df_crypto = comm_crypto[cols]
df_cryto.to_csv('crypto.csv')
df_crypto = df_crypto.dropna()
df_crypto = df_crypto.drop_duplicates().reset_index()
df_crypto = df_crypto.drop(columns='index')
df_crypto.head(10)

,author,selftext,retrieved_on,score
0,PineappleFund,Hello! \n\nI remember starting at bitcoin a fe...,1513218630,15363
1,EaFaer,"On November 28, 2017, the US Senate, Committee...",1512791925,12358
2,jhamel2,,1521511199,4008
3,amorpisseur,,1515810204,3738
4,ivanraszl,,1551376037,1116
5,OurLordAndSaviorSam,He talked the class through the entire concept...,1516374140,588
6,ecurrencyhodler,,1523039102,543
7,Tutellus-io,,1517451154,488
8,IncognitoVito0,,1562178274,482
9,vnielz,,1562598705,472


## Scraping Stocks related News

In [17]:
start = int(time.mktime(parse('2016-2-1 8:00:00').timetuple()))
end = int(time.mktime(parse('2020-2-24 8:00:00').timetuple()))
print(start, end)
stocks = get_submits('finance', 'stocks', end, start)

1454335200 1582552800
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [69]:
comm_stocks = pd.DataFrame(stocks)
print(comm_stocks.shape)
print(comm_stocks.info)
comm_stocks.head()

(7422, 73)
<bound method DataFrame.info of      all_awardings allow_live_comments               author  \
0               []               False  AnotherDullUsername   
1               []               False           Qwerty1458   
2               []               False              ferchud   
3              NaN                 NaN       hellow_friends   
4               []                 NaN       coolcomfort123   
...            ...                 ...                  ...   
7417            []               False              Davecvs   
7418            []               False              Davecvs   
7419            []               False              Cube189   
7420            []               False              Cube189   
7421            []               False          Mr_Danzilla   

     author_flair_css_class author_flair_richtext author_flair_text  \
0                      None                    []              None   
1                      None                    []         

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,can_mod_post,...,media_embed,secure_media_embed,suggested_sort,author_id,rte_mode,awarders,og_description,og_title,author_premium,removed_by_category
0,[],False,AnotherDullUsername,None,[],None,text,t2_181m9be9,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Qwerty1458,None,[],None,text,t2_17hxme,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,ferchud,None,[],None,text,t2_3wugo47l,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,hellow_friends,None,NaN,None,NaN,t2_futu9,NaN,NaN,...,{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],NaN,coolcomfort123,None,[],None,text,t2_2sj2vd1h,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
cols = ['author', 'selftext', 'retrieved_on', 'score']
df_stocks = comm_stocks[cols]
df_stocks.to_csv('stocks.csv')
df_stocks = df_stocks.dropna()
df_stocks = df_stocks.drop_duplicates().reset_index()
df_stocks = df_stocks.drop(columns='index')
df_stocks.head(10)

,author,selftext,retrieved_on,score
0,AnotherDullUsername,I've created a semi-automated spreadsheet that...,1565707199,755
1,Qwerty1458,"I mean if people who are extremely smart, go t...",1565832097,752
2,ferchud,[https://www.reuters.com/article/us-facebook-c...,1568543003,616
3,hellow_friends,Hello /r/stocks. I am a college student who ha...,1489424137,439
4,coolcomfort123,https://finance.yahoo.com/news/bank-of-america...,1561697762,421
5,sentdex,"Hi there, my name is Harrison and I frequently...",1489514365,371
6,bobbythehermit,If you had copied Shkreli's portfolio from his...,1531703393,347
7,baycommuter,I've been in the market for more than 30 years...,1565052366,279
8,InterestingNews1,Value Stocks\n\nWestern Union presented a new ...,1569674461,220
9,spaceboii1,"I am a 21 year old finance student, looking to...",1567372115,175


## Scraping News Relating to Gold

In [102]:
start = int(time.mktime(parse('2016-2-1 8:00:00').timetuple()))
end = int(time.mktime(parse('2020-2-24 8:00:00').timetuple()))
print(start, end)
gold = get_submits('finance', 'gold', end, start)

1454335200 1582552800
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [111]:
comm_gold = pd.DataFrame(gold)
print(comm_gold.shape)
print(comm_gold.info)
comm_gold.head()

(5193, 65)
<bound method DataFrame.info of              author author_flair_css_class author_flair_text brand_safe  \
0            bjtbtc                   None              None      False   
1       goldiraplan                   None              None        NaN   
2       goldiraplan                   None              None        NaN   
3       goldiraplan                   None              None        NaN   
4       goldiraplan                   None              None        NaN   
...             ...                    ...               ...        ...   
5188  LightningPlus                   None              None        NaN   
5189  LightningPlus                   None              None        NaN   
5190  LightningPlus                   None              None        NaN   
5191  LightningPlus                   None              None        NaN   
5192  LightningPlus                   None              None        NaN   

     can_mod_post  contest_mode  created_utc            

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,full_link,id,...,total_awards_received,updated_utc,awarders,author_premium,crosspost_parent,crosspost_parent_list,post_hint,preview,thumbnail_height,thumbnail_width
0,bjtbtc,None,None,False,False,False,1516212739,self.Gold,https://www.reddit.com/r/Gold/comments/7r2y0v/...,7r2y0v,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,goldiraplan,None,None,NaN,NaN,False,1482426845,economictimes.indiatimes.com,https://www.reddit.com/r/Gold/comments/5jrp7a/...,5jrp7a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,goldiraplan,None,None,NaN,NaN,False,1482512648,zerohedge.com,https://www.reddit.com/r/Gold/comments/5jy3zb/...,5jy3zb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,goldiraplan,None,None,NaN,NaN,False,1482338959,economictimes.indiatimes.com,https://www.reddit.com/r/Gold/comments/5jkxgd/...,5jkxgd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,goldiraplan,None,None,NaN,NaN,False,1483116995,marketwatch.com,https://www.reddit.com/r/Gold/comments/5l4f0s/...,5l4f0s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
cols = ['author', 'selftext', 'retrieved_on', 'score']
df_gold = comm_gold[cols]
df_gold.to_csv('gold.csv')
df_gold = df_gold.dropna()
df_gold = df_gold.drop_duplicates().reset_index()
df_gold = df_gold.drop(columns='index')
df_gold.head(10)

,author,selftext,retrieved_on,score
0,bjtbtc,"Hello Gold members,\n\nAs a young and inexperi...",1516266497,2
1,goldiraplan,,1484430926,1
2,goldiraplan,,1484434401,1
3,goldiraplan,,1484427006,1
4,goldiraplan,,1484457196,1
5,goldiraplan,,1489466179,1
6,goldiraplan,,1489452233,1
7,goldiraplan,,1489427815,1
8,goldiraplan,,1489403590,1
9,goldiraplan,,1489420696,1


## Scraping Bitcoin related News

In [108]:
start = int(time.mktime(parse('2016-2-1 8:00:00').timetuple()))
end = int(time.mktime(parse('2020-2-24 8:00:00').timetuple()))
print(start, end)
bitcoin = get_submits('finance', 'bitcoin', end, start)

1454335200 1582552800
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [109]:
comm_bit = pd.DataFrame(bitcoin)
print(comm_bit.shape)
print(comm_bit.info)
comm_bit.head()

(7405, 78)
<bound method DataFrame.info of                   author author_flair_css_class    author_flair_text  \
0          PineappleFund                   noob  redditor for 1 week   
1                 EaFaer                   None                 None   
2                jhamel2                   None                 None   
3            amorpisseur                   None                 None   
4              ivanraszl                   None                 None   
...                  ...                    ...                  ...   
7400         nugget_alex                   None                 None   
7401  gokumarketofficial                   None                 None   
7402         nugget_alex                   None                 None   
7403  gokumarketofficial                   None                 None   
7404  gokumarketofficial                   None                 None   

     brand_safe  can_mod_post  contest_mode  created_utc        domain  \
0          True   

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,edited,full_link,...,allow_live_comments,total_awards_received,author_flair_template_id,awarders,steward_reports,og_description,og_title,media_metadata,author_premium,removed_by_category
0,PineappleFund,noob,redditor for 1 week,True,False,False,1513170057,self.Bitcoin,1.513174e+09,https://www.reddit.com/r/Bitcoin/comments/7jj0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EaFaer,None,None,True,False,False,1512739690,self.Bitcoin,NaN,https://www.reddit.com/r/Bitcoin/comments/7iey...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jhamel2,None,None,True,False,False,1521463923,twitter.com,NaN,https://www.reddit.com/r/Bitcoin/comments/85j8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,amorpisseur,None,None,True,False,False,1515744927,twitter.com,NaN,https://www.reddit.com/r/Bitcoin/comments/7pv8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ivanraszl,None,None,NaN,False,False,1551376036,i.redd.it,NaN,https://www.reddit.com/r/Bitcoin/comments/avtx...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
cols = ['author', 'selftext', 'retrieved_on', 'score']
df_bit = comm_bit[cols]
df_bit.to_csv('bitcoin.csv')
df_bit = df_bit.dropna()
df_bit = df_bit.drop_duplicates().reset_index()
df_bit = df_bit.drop(columns='index')
df_bit.head(10)

,author,selftext,retrieved_on,score
0,PineappleFund,Hello! \n\nI remember starting at bitcoin a fe...,1513218630,15363
1,EaFaer,"On November 28, 2017, the US Senate, Committee...",1512791925,12358
2,jhamel2,,1521511199,4008
3,amorpisseur,,1515810204,3738
4,ivanraszl,,1551376037,1116
5,OurLordAndSaviorSam,He talked the class through the entire concept...,1516374140,588
6,ecurrencyhodler,,1523039102,543
7,Tutellus-io,,1517451154,488
8,IncognitoVito0,,1562178274,482
9,vnielz,,1562598705,472
